In [1]:
import os
import numpy as np
import glob
import PIL.Image as Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import models
from torchsummary import summary
import torch.optim as optim
from sklearn.model_selection import KFold
from time import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
import sys
import os
import skorch
from skorch import NeuralNet
import warnings

# Suppress only FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.model_selection import GridSearchCV

sys.path.append('/zhome/45/0/155089/Deeplearning_in_computer_vision/Segmentation_project/Asignments_DeepLearningForCV/')  
from Performance_Metrics import dice_coefficient, intersection_over_union, accuracy, sensitivity, specificity
#import dataset PH2 
from DataLoader_PH2 import train_loader, val_loader, test_loader, Full_set, PH2
import time 
from time import time  # Correct import

200


In [24]:
from skorch.dataset import Dataset
from sklearn.metrics import accuracy_score
import numpy as np
from skorch.dataset import unpack_data


def accuracy_scoring(net, X, y=None, sample_weight=None):
    """Calculate accuracy score for the net."""
    if sample_weight is not None:
        raise NotImplementedError(
            "sample_weight for accuracy_scoring is not yet supported."
        )

    net.check_is_fitted()

    dataset = net.get_dataset(X, y)
    iterator = net.get_iterator(dataset, training=False)

    all_predictions = []
    all_targets = []

    for batch in iterator:
        yp = net.evaluation_step(batch, training=False)  # Model predictions
        yi = unpack_data(batch)[1]  # Ground truth labels

        # If yp is logits or probabilities, apply threshold to convert to binary (for binary segmentation)
        # For multi-class segmentation, you may need to use argmax, depending on your task
        if yp.shape[-1] > 1:
            yp = yp.argmax(dim=1)  # Multi-class segmentation
        else:
            yp = (yp > 0.5).float()  # Binary segmentation (thresholding)

        # Convert tensors to numpy arrays
        yp = yp.detach().cpu().numpy()
        yi = yi.detach().cpu().numpy()

        # Reshape predictions and targets to 1D arrays for pixel-wise accuracy calculation
        yp = yp.reshape(-1)
        yi = yi.reshape(-1)

        all_predictions.append(yp)
        all_targets.append(yi)

    # Concatenate all predictions and targets
    all_predictions = np.concatenate(all_predictions, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)

    # Calculate and return accuracy
    return accuracy_score(all_targets, all_predictions)


def accuracy(y_true, y_pred):
    # Check if the inputs are PyTorch tensors or NumPy arrays
    if isinstance(y_pred, torch.Tensor) and isinstance(y_true, torch.Tensor):
        y_pred = (y_pred > 0.5).float()  # Thresholding predictions for PyTorch
        correct = (y_true == y_pred).float().sum()  # Correct predictions in PyTorch
        return (correct / y_true.numel()).item()  # Return scalar accuracy for PyTorch

    elif isinstance(y_pred, np.ndarray) and isinstance(y_true, np.ndarray):
        y_pred = (y_pred > 0.5).astype(float)  # Thresholding predictions for NumPy
        correct = np.sum(y_true == y_pred)  # Correct predictions in NumPy
        return correct / y_true.size  # Return accuracy for NumPy

    else:
        raise TypeError("Expected both inputs to be either PyTorch tensors or NumPy arrays")

class LossWrapper(nn.Module):
    def __init__(self, loss_fn):
        super(LossWrapper, self).__init__()
        self.loss_fn = loss_fn

    def forward(self, y_real, y_pred):
        return self.loss_fn(y_real, y_pred)
    
class CustomNeuralNet(NeuralNet):
    def initialize_criterion(self):
        if isinstance(self.criterion, nn.Module):
            self.criterion_ = self.criterion
        else:
            # Initialize the wrapped loss function as a criterion
            self.criterion_ = LossWrapper(self.criterion)
        return self
    
    def score(self, X, y=None):
        if y is not None and isinstance(y, torch.Tensor):
            y = y.numpy()  # Ensure y is a numpy array
        return accuracy_scoring(self, X, y)





In [29]:
from Encoder_Decoder_PH2 import EncDec
from functools import partial
from UNet2 import bce_loss, dice_loss, focal_loss, bce_total_variation, focal_loss_chatten, stable_bce_loss
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Wrapping your custom loss functions using functools.partial
param_grid = {
    'criterion': [
    dice_loss, 
    # focal_loss, 
    bce_total_variation, 
    # focal_loss_chatten, 
    # stable_bce_loss
],
    # 'max_epochs':[15,
    #                 20,
    #                 25,
    #                 30],
    # 'batch_size': [10,
    #                20,
    #                30,
    #                40]
              }

# Initialize model
model = CustomNeuralNet(EncDec, 
                  optimizer=optim.Adam, 
                  verbose=1,
                  max_epochs=20,
                  batch_size=40,
                #   score = dice_coefficient,
                  criterion=bce_loss,  # A placeholder, will be overwritten in GridSearchCV
                  device=device,
                  iterator_train__shuffle=False,
)



# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=5,)

# Assuming you have a dataset called PH2

train_transform = transforms.Compose([transforms.Resize((128, 128)), transforms.ToTensor()])
Full_set = PH2(transform=train_transform)

X = torch.stack([X for X,_ in Full_set])
y = torch.stack([y for X,y in Full_set])


# dataset = CustomDataset(Full_set)
# # Fit grid search
grid_result = grid.fit(X,y)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        0.2499       -0.2272  0.1204
      2        0.0267       -0.2249  0.1007
      3       -0.1437       -0.2352  0.0976
      4       -0.2404       -0.3290  0.0881
      5       -0.3103       -0.3781  0.0859
      6       -0.3635       -0.4008  0.0855
      7       -0.4166       -0.4201  0.0857
      8       -0.4738       -0.4269  0.0857
      9       -0.5564       -0.4658  0.0856
     10       -0.7268       -0.5100  0.0856
     11       -2.4703       -0.5402  0.0859
     12        0.8937       -0.1832  0.0857
     13       -1.6911       -0.1917  0.0854
     14        0.5567       -0.2401  0.0853
     15        0.4008       -0.3298  0.0860
     16        0.2306       -0.5549  0.0853
     17        0.1154        1.5784  0.0855
     18        0.0271       -0.2038  0.0851
     19       -0.0554       -0.2874  0.0856
     20       -0.1314       -0.2925  0.0854
  epoch    train_loss    valid_l

In [30]:
grid.best_params_

{'criterion': <function UNet2.dice_loss(y_real, y_pred)>}

In [31]:
grid.best_score_


np.float64(0.6776934814453125)

In [33]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.677693 using {'criterion': <function dice_loss at 0x7f7cc3465a80>}
0.677693 (0.096186) with: {'criterion': <function dice_loss at 0x7f7cc3465a80>}
0.677693 (0.096186) with: {'criterion': <function bce_total_variation at 0x7f7cc3464f40>}
